In [5]:
import pandas
from bs4 import BeautifulSoup
import  os
import json
import re
import urllib as urllib2
from urllib import request
import requests
from time import sleep
from selenium import webdriver


In [6]:
BASE_URL = 'http://www.reformagkh.ru'
EXTENDED_BASE = 'https://www.reformagkh.ru/myhouse'
AREAS_URL = 'https://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax'
AREAS_FILE = 'regions.json'
MUN_URLPAT = 'http://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax&tid=%s'
MUN_FILEPAT = 'municipal.json'
BUILDING_LIST_PAT = 'http://www.reformagkh.ru/myhouse/list?tid=%s&sort=alphabet&item=tp&mkdsort=name&mkdorder=asc&perpage=10000'
BUILDING_URLPAT = 'http://www.reformagkh.ru/myhouse/view/%s/?group=0'
BUILDING_KEYS = ['house_id', 'url', 'address','year', 'area', 'num_people', 'status']

In [7]:
def table2dict(tab, skip=0, keys=[], keyshift=2):
    rows = []
    i = 0
    for k in tab.findAll('tr', recursive=False):
        i += 1
        if i - skip <= 0: continue
        row = {}
        #print k
        tds = k.findAll('td')
        url=tds[0].find('a')['href']
        row['url']=BASE_URL+url
        row['house_id']=re.search('/\d+/',url).group(0).replace('/','')
        tdslen = len(tds)
        for n in range(0, len(keys), 1):
            try:
                row[keys[n+keyshift]] = tds[n].text
            except:
                pass
        #print row
        rows.append(row)
    return rows

In [8]:
def extract_areas():
    """Extracts indicators data by area code. Saves as JSON file"""
    filename = AREAS_FILE
    if os.path.exists(filename): return None
    print("Processing Areas", AREAS_URL)
    
    url = AREAS_URL
    browser = webdriver.PhantomJS(executable_path="C:\\Users\\772\\Documents\\phantomjs-2.1.1-windows\\bin\\phantomjs.exe")
    browser.get(url)
    sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    columns = soup.find_all('div', attrs={'class':'col-4 py-4'})
    links = []
    for i in columns:
        for a in i.find_all('a', href=True):
            print("Found the URL:", a['href'])
            links.append({"link": EXTENDED_BASE + a['href'], "area_name": a.text})
      
    f = open(filename, 'w')
    f.write(json.dumps(links, indent=4, ensure_ascii=False))
    f.close()
    print("Processing Areas", 'written')
    return links


In [9]:
extract_areas()

In [39]:
def extract_municipal():
    """Extracts data about municipal areas"""
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    js = json.loads(data)
    print(js)
    links = []

    browser = webdriver.Chrome("C:\\Users\\772\\Downloads\\\chromedriver_win32\\chromedriver.exe")
    #browser = webdriver.Chrome(executable_path="C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe") 
    #browser = webdriver.PhantomJS(executable_path="C:\\Users\\772\\Docchromedriver.exeuments\\phantomjs-2.1.1-windows\\bin\\phantomjs.exe")
    for area in js: 
        print("обхожу районы", area['area_name'])
        browser.get(area['link'])
        sleep(3)
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        columns = soup.find_all('div', attrs={'class':'col-4 py-4'})
        for i in columns:
            for a in i.find_all('a', href=True):
                print("Found the URL:", EXTENDED_BASE + a['href'], a.text)
                links.append({"link": EXTENDED_BASE + a['href'], "munuicipal_name": a.text, "area_name": area['area_name'] })
                
    f = open(MUN_FILEPAT, 'w')
    f.write(json.dumps(links, indent=4, ensure_ascii=False))
    f.close()
    print("Processing Areas", 'written')
    return links        

In [38]:
extract_municipal()

[{'link': 'https://www.reformagkh.ru/myhouse?tid=2362349', 'area_name': 'Восточный'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2283381', 'area_name': 'Город Московский'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362348', 'area_name': 'Западный'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362347', 'area_name': 'Зеленоградский'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2283387', 'area_name': 'Мосрентген'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362339', 'area_name': 'Новомосковский'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362346', 'area_name': 'Северный'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362345', 'area_name': 'Северо-Восточный'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362344', 'area_name': 'Северо-Западный'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2283392', 'area_name': 'Сосенское'}, {'link': 'https://www.reformagkh.ru/myhouse?tid=2362338', 'area_name': 'Троицкий'}, {'link': 'https://www.reformagkh.ru/

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

...

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

<html lang="en"><head>
<!-- Required meta tags -->
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<title>    Мой дом
 | Реформа ЖКХ</title>
<meta content="width=1060, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="/images/favicon.ico" rel="icon"/>
<link href="/images/favicon.png" rel="shortcut icon" type="image/png"/>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/favicon-144x144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>
<link href="/images/favicon-114x114.png" rel="apple-touch-icon-precomposed" sizes="114x114"/>
<link href="/images/favicon-72x72.png" rel="apple-touch-icon-precomposed" sizes="72x72"/>
<link href="/images/favicon-57x57.png" rel="apple-touch-icon-precomposed"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,600,700&amp;display=swap" rel="stylesheet"/>
<link href="/assets/vendor.bc2bf8.css" rel="stylesheet" type="text/css"/>
<link href="/a

[{'link': 'https://www.reformagkh.ru/myhouse?tid=2281093',
  'munuicipal_name': 'муниципальный округ Богородское',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281094',
  'munuicipal_name': 'муниципальный округ Вешняки',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281095',
  'munuicipal_name': 'муниципальный округ Восточное Измайлово',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281096',
  'munuicipal_name': 'муниципальный округ Восточный',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281097',
  'munuicipal_name': 'муниципальный округ Гольяново',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281098',
  'munuicipal_name': 'муниципальный округ Ивановское',
  'area_name': 'Восточный'},
 {'link': 'https://www.reformagkh.ru/myhouse?tid=2281099',
  'munuicipal_name': 'муниципальный округ Измайлово',
  'area_name': 'Восточны

In [58]:
def get_buildings_list(url = None):
    if url is None: 
        f = open(MUN_FILEPAT, 'r')
        data = f.read()
        f.close()
        js = json.loads(data)
        print(js)
        url = []
        for link in js: 
            url.append(link['link'])
    else:
        url = [url]
    links = []
    browser = webdriver.Chrome("C:\\Users\\772\\Downloads\\\chromedriver_win32\\chromedriver.exe")
    
    for link in url:
        browser.get(link)
        sleep(3)
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        columns = soup.find_all('table', attrs={'class':'table text-center mb-0'})
        rows = soup.find_all('tr')
        return rows

In [59]:
rows = get_buildings_list('https://www.reformagkh.ru/myhouse?tid=2281077')

In [60]:
rows[2]

<tr>
<td class="text-left"><a href="/myhouse/profile/view/7706328/">г. Москва, ул. Академика Пилюгина, д. 6</a></td>
<td>2008</td>
<td>36 863,00</td>
<td class="text-right">ООО УК «Свитхом»</td>
<td class="text-left position-relative">
<a data-favorite-house-id="7706328" data-is-favorite="" data-progress="0" data-user-id="" href="#" title="Добавить дом в избранное">
<img height="20" src="/images/stars/star_off.svg"/>
</a>
</td>
</tr>

In [61]:
def save_buildings(tid =None):
    """Saves buildings data by TID"""
    filename = 'data/raw/buildings_%s.json' % 'lomonosovsky'
    #if os.path.exists(filename):
    #    print "Skipping buildings for mun %s" % tid
    #    return
    #print "Processing buildings for mun %s" % tid
    url = 'https://www.reformagkh.ru/myhouse?tid=2281077&item=tp'
    #url = BUILDING_LIST_PAT % tid
    rows, lastp =  get_buildings_list(url, getlast=False)
    #print '- processed', url
    f = open(filename, 'w')
    json.dump(rows, f, indent=4)
    f.close()

In [ ]:
def get_buildings_list():
    

In [62]:
save_buildings()

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
def extract_buildings_list():
    """Extracts list of buildings for each area"""
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    areas = json.loads(data)
    for area in areas:
        f = open(MUN_FILEPAT % area['tid'], 'r')
        data = f.read()
        f.close()
        try:
            muns = json.loads(data)
        except:
            continue
        for mun in muns:
            save_buildings(mun['tid'])
        save_buildings(area['tid'])


In [ ]:

def get_buildings_data(tid):
    """Collect buildings data for specific area by TID (territory id)"""
    filename = 'data/raw/buildings_%s.json' % tid
    f = open(filename, 'r')
    rows = json.load(f)
    f.close()
    return rows

In [ ]:

def refine_buildings():
    """Prepare data for publishing"""
    idkeys = []
    allrows = []
    allterr = []
    f = open(AREAS_FILE, 'r')
    data = f.read()
    f.close()
    areas = json.loads(data)
    for area in areas:
        f = open(MUN_FILEPAT % area['tid'], 'r')
        data = f.read()
        f.close()
        try:
            muns = json.loads(data)
        except:
            continue
        for mun in muns:
            allterr.append({'name' : mun['name'], 'id' : mun['tid']})
            rows = get_buildings_data(mun['tid'])
            for row in rows:
                if row['house_id'] not in idkeys: idkeys.append(row['house_id'])
                else: continue
                item = {}
                item['area_name'] = mun['name']
                item['area_id'] = mun['tid']
                item['address'] = row['address']
                item['house_year'] = row['year']
                item['house_id'] = row['house_id']
                item['house_area'] = row['area']
                item['num_people'] = row['num_people']
                allrows.append(item)
        rows = get_buildings_data(area['tid'])
        if len(rows) > 0:
            allterr.append({'name' : area['name'], 'id' : area['tid']})
        for row in rows:
            if row['house_id'] not in idkeys: idkeys.append(row['house_id'])
            else: continue
            item = {}
            item['area_name'] = mun['name']
            item['area_id'] = mun['tid']
            item['address'] = row['address']
            item['house_year'] = row['year']
            item['house_id'] = row['house_id']
            item['house_area'] = row['area']
            item['num_people'] = row['num_people']
            allrows.append(item)
    f = open('data/refine/buildings.json', 'w')
    json.dump(allrows, f, indent=4)
    f.close()
    f = open('data/refine/areas.json', 'w')
    json.dump(allterr, f, indent=4)
    f.close()


Processing Areas http://www.reformagkh.ru/myhouse?tid=2280999&sort=alphabet&item=mkd&ajax
<html><head><script lang="text/javascript">
    var checkKey = 'a0ee50584a39e6d7c4e6e3191b2cf8dd',
        key1 = '7b2808d07931224c27054f522a3395d15b3100ea640576bcc08f',
        val1 = Number('1060883614'),
        key2 = '7b2808d07931224c27054f522a3395d13ee9da1dd87b63c6f83c',
        val2 = Number('1351839011');

    var xhrRequest = new XMLHttpRequest();

    xhrRequest.open('POST', '/save-check-data');
    xhrRequest.setRequestHeader('Content-Type', 'application/x-www-form-urlencoded');
    xhrRequest.setRequestHeader('X-Requested-With', 'XMLHttpRequest');
    xhrRequest.onload = function () {
        var response = JSON.parse(xhrRequest.responseText || '{}');

        if (xhrRequest.status === 200 && !!response.success) {
            window.location.reload();
        } else {
            window.location.href = '/';
        }
    };
    xhrRequest.send(encodeURI('key1=' + key1 + '&key2=' + key2

AttributeError: 'NoneType' object has no attribute 'find_all'